This script edits model entries in the database to add or modify the publication field for a provided list of model.

In [1]:
from dotenv import load_dotenv
import os
from elasticsearch import Elasticsearch  # Base function for interacting with Elasticsearch
from load_ini import *

# remove excessive HTTPS request warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# connect to Elasticsearch
load_dotenv()
client = Elasticsearch("https://localhost:9200/", api_key=os.getenv('API_KEY'),verify_certs=False)

# index name
INDEX_NAME = "wind"

# get list of documents to udpate
DIR_NAME = os.getcwd()
FILE_NAME = "pubs.csv"
FILE_PATH = os.path.join(DIR_NAME, FILE_NAME)
models, _ = read_csv(FILE_PATH)

for m in models:
    q = {
        "script": {
            "source": f"ctx._source.Publication='{m[0]}'",  # update publication field
            "lang": "painless"
        },
        "query": {
            "match": {
                'Model name': m[1]
            }
        }
    }
    client.update_by_query(body=q, index='wind')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/elasticsearch/_sync/client/__init__.py:400: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
